# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [8]:
df = pd.read_csv('previsao_de_renda.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [38]:
# 1)
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Verificar correlação entre as variáveis independentes
correlation_matrix = train_df.corr()
print(correlation_matrix)

# Verificar valores NaN em dados de treinamento
print(train_df.isnull().sum())

                       Unnamed: 0  id_cliente  posse_de_veiculo  \
Unnamed: 0               1.000000    0.021388         -0.002764   
id_cliente               0.021388    1.000000         -0.012814   
posse_de_veiculo        -0.002764   -0.012814          1.000000   
posse_de_imovel          0.004875   -0.100945         -0.016380   
qtd_filhos              -0.007380    0.018388          0.117046   
idade                   -0.000877   -0.053468         -0.170954   
tempo_emprego           -0.000296   -0.034427         -0.068483   
qt_pessoas_residencia   -0.008898    0.027350          0.166855   
renda                    0.005948   -0.001645          0.103745   

                       posse_de_imovel  qtd_filhos     idade  tempo_emprego  \
Unnamed: 0                    0.004875   -0.007380 -0.000877      -0.000296   
id_cliente                   -0.100945    0.018388 -0.053468      -0.034427   
posse_de_veiculo             -0.016380    0.117046 -0.170954      -0.068483   
posse_de_imov

Como a variavel tempo_emprego possui valores nulos, ela será removida

In [41]:
# 2)
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Loop para ajustar o modelo com diferentes valores de alpha e avaliar o R-quadrado na base de testes
for alpha in alphas:
    modelo = 'np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade'
    md = smf.ols(modelo, data=train_df)
    reg = md.fit_regularized(method = 'elastic_net' ,refit=True, L1_wt=0, alpha=alpha)

    # Avaliando o modelo na base de testes
    y_pred = reg.predict(test_df)
    
    # Verificar se há valores NaN nos dados previstos
    if np.isnan(y_pred).any():
        print(f"Alpha={alpha}: Os valores previstos contêm NaNs.")
        continue
    
    # Verificar se há valores NaN nos dados de teste
    if test_df['renda'].isnull().any():
        print(f"Alpha={alpha}: Os dados de teste contêm valores NaN.")
        continue
    
    r2 = r2_score(test_df['renda'], np.exp(y_pred))  # Convertendo de log-renda para renda original
    print(f"Alpha={alpha}: R²={r2}")

Alpha=0: R²=0.023051360512166275
Alpha=0.001: R²=0.02480856010809729
Alpha=0.005: R²=0.022929248245798672
Alpha=0.01: R²=-0.044609316600812576
Alpha=0.05: R²=-394.6946416599423
Alpha=0.1: R²=-8275.723607897213


O melhor modelo seria o com Alpha=0.001

In [43]:
# 3)
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
for alpha in alphas:
    modelo = 'np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego'
    md = smf.ols(modelo, data=train_df)
    reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = alpha)
    print(f'### Alpha: {alpha}')
    print(reg.summary())
    print(f'\n~~~')

### Alpha: 0
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     696.6
Date:                Mon, 12 Feb 2024   Prob (F-statistic):               0.00
Time:                        19:22:00   Log-Likelihood:                -10267.
No. Observations:                9313   AIC:                         2.055e+04
Df Residuals:                    9306   BIC:                         2.061e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept  

### Alpha: 0.1
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.340
Method:                 Least Squares   F-statistic:                     1201.
Date:                Mon, 12 Feb 2024   Prob (F-statistic):               0.00
Time:                        19:22:01   Log-Likelihood:                -10291.
No. Observations:                9313   AIC:                         2.059e+04
Df Residuals:                    9309   BIC:                         2.063e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept

In [34]:
# 4)
test_df.loc[test_df['sexo'] == 'F', 'fem'] = 1
test_df.loc[test_df['sexo'] == 'M', 'fem'] = 0

X = test_df[['fem', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade']]
y = test_df['renda'].astype(float)

reg_stepwise = sm.OLS(y, sm.add_constant(X.astype(int))).fit()
reg_stepwise.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     80.94
Date:                Mon, 12 Feb 2024   Prob (F-statistic):           6.77e-81
Time:                        19:15:31   Log-Likelihood:                -38926.
No. Observations:                3750   AIC:                         7.786e+04
Df Residuals:                    3744   BIC:                         7.790e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             5534.9817    636.262      8.699      0.000    4287.529    6782.435
fem              -5518.7201    293.728    -18.789      0.000   -6094.602   -4942.838
posse_de_veiculo   -55.9498    280.192     -0.200      0.842    -605.294     493.395
posse_de_imovel    592.3208    275.685      2.149      0.032      51.813    1132.829
qtd_filhos         471.7556    180.665      2.611      0.009     117.544     825.967
idade               78.0282     12.699      6.145      0.000      53.131     102.925
==============================================================================
Omnibus:                     4988.641   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1640110.471
Skew:                           7.285   Prob(JB):                         0.00
Kurtosis:                     104.412   Cond. No.                         232.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#5)
'''
O os parâmetros possuem alteraçao de p-value entre os modelos, porém o modelo de melhor desempenho foi o de regressão LASSO
'''

In [71]:
#6) 
modelo = 'np.log(renda) ~ C(sexo) + posse_de_imovel + tempo_emprego'
md = smf.ols(modelo, data=test_df)
reg = md.fit_regularized(method='elastic_net', refit=True, L1_wt=1, alpha=0)
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     422.4
Date:                Mon, 12 Feb 2024   Prob (F-statistic):          4.83e-291
Time:                        19:28:16   Log-Likelihood:                -3409.3
No. Observations:                3114   AIC:                             6829.
Df Residuals:                    3110   BIC:                             6859.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

Removi algumas variveis que estavam com valor elevado de p-value para esse modelo

In [76]:
#7)

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

train_df.loc[train_df['sexo'] == 'F', 'fem'] = 1
train_df.loc[train_df['sexo'] == 'M', 'fem'] = 0

test_df.loc[test_df['sexo'] == 'F', 'fem'] = 1
test_df.loc[test_df['sexo'] == 'M', 'fem'] = 0

# # Separar variáveis independentes e dependentes
X_train = train_df[['fem', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade']]
y_train = np.log(train_df['renda'])  # Usar log-renda como variável dependente

X_test = test_df[['fem', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade']]
y_test = np.log(test_df['renda'])  # Usar log-renda como variável dependente

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)

# Fazer previsões
y_pred_train = tree_reg.predict(X_train)
y_pred_test = tree_reg.predict(X_test)

# Avaliar o desempenho do modelo
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"R² (treinamento): {r2_train}")
print(f"R² (teste): {r2_test}")

R² (treinamento): 0.28871301411795847
R² (teste): 0.18233133539507862
